<a href="https://colab.research.google.com/github/tintinap/Genetic-Algorithm/blob/main/genetic_algorithm_breast_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from itertools import combinations
import pandas as pd
import numpy as np
import random
import ast

import mysql.connector

In [4]:
#breast_cancer
# url = 'https://github.com/tintinap/Genetic-Algorithm/blob/main/dataset/diabetes.csv?raw=true'
df = pd.read_csv('./dataset/breast_cancer.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
#turn M = 1 B = 0
df["diagnosis"] = df['diagnosis'].apply(lambda i : 1 if i == 'M' else 0)
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [7]:
features = df.columns.tolist()[2:-1]
features

['radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'fractal_dimension_mean',
 'radius_se',
 'texture_se',
 'perimeter_se',
 'area_se',
 'smoothness_se',
 'compactness_se',
 'concavity_se',
 'concave points_se',
 'symmetry_se',
 'fractal_dimension_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst',
 'fractal_dimension_worst']

In [8]:
X = df[features]
label = 'diagnosis'
y = df[label]

# normalize
X = X.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595


<h1> Fitness Function

<h2> Manhattan Score Table

In [10]:
df_md = {'Outcome': []}

for f in features:
    df_md['Outcome'].append(np.mean([np.sqrt(abs(a[0]-a[1])) for a in zip(X[f], y)]))

df_md = pd.DataFrame.from_dict(df_md)
df_md.index = features
df_md.T

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
Outcome,0.563995,0.553759,0.558932,0.441643,0.619624,0.486704,0.409237,0.452685,0.605469,0.50061,...,0.522842,0.586557,0.509982,0.384911,0.623639,0.442786,0.427644,0.595616,0.499485,0.415662


<h1>Function Tools

In [11]:
def convertZeroOne(features, zeroOneList):
    """
    convert list of 0 and 1 to be list of string feature
    """
    return [features[i] for i in range(len(zeroOneList)) if zeroOneList[i] == 1]

<h1>Set up db

In [12]:
def checkTableExists(dbcon, tablename):
    dbcur = dbcon.cursor()
    dbcur.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(tablename.replace('\'', '\'\'')))
    if dbcur.fetchone()[0] == 1:
        dbcur.close()
        return True

    dbcur.close()
    return False

In [13]:
def createTable(dbcon, tablename, create_sql):
    if checkTableExists(dbcon, tablename):
        mycursor.execute('drop TABLE', tablename)      
    mycursor.execute(create_sql)


In [21]:
mydb = mysql.connector.connect(
  host="localhost",
  user="admin",
  password="admin",
  database='ml_db'
)
mycursor = mydb.cursor(buffered=True)

print(mydb)

In [33]:
feature_create_sql = ''
for feature in features:
    feature_create_sql  += str(feature).replace(' ', '_')+' int(1) ,'
feature_create_sql = feature_create_sql[:-1]
feature_create_sql

'radius_mean int(1) ,texture_mean int(1) ,perimeter_mean int(1) ,area_mean int(1) ,smoothness_mean int(1) ,compactness_mean int(1) ,concavity_mean int(1) ,concave_points_mean int(1) ,symmetry_mean int(1) ,fractal_dimension_mean int(1) ,radius_se int(1) ,texture_se int(1) ,perimeter_se int(1) ,area_se int(1) ,smoothness_se int(1) ,compactness_se int(1) ,concavity_se int(1) ,concave_points_se int(1) ,symmetry_se int(1) ,fractal_dimension_se int(1) ,radius_worst int(1) ,texture_worst int(1) ,perimeter_worst int(1) ,area_worst int(1) ,smoothness_worst int(1) ,compactness_worst int(1) ,concavity_worst int(1) ,concave_points_worst int(1) ,symmetry_worst int(1) ,fractal_dimension_worst int(1) '

In [35]:
#bc_population
bc_population_create_sql = 'create table bc_POPULATION ('+ feature_create_sql +');'
try:
    createTable(mydb, 'bc_population', bc_population_create_sql)
    print('bc_population created')
except:
    mycursor.execute('drop table bc_population')
    createTable(mydb, 'bc_population', bc_population_create_sql)
    print('bc_population created')

#bc_selection
bc_selection_create_sql = 'create table bc_selection ('+ feature_create_sql +');'
try:
    createTable(mydb, 'bc_selection', bc_selection_create_sql)
    print('bc_selection created')
except:
    mycursor.execute('drop table bc_selection')
    createTable(mydb, 'bc_selection', bc_selection_create_sql)
    print('bc_selection created')

#bc_crossover
bc_crossover_create_sql = 'create table bc_crossover ('+ feature_create_sql +');'
try:
    createTable(mydb, 'bc_crossover', bc_crossover_create_sql)
    print('bc_crossover created')
except:
    mycursor.execute('drop table bc_crossover')
    createTable(mydb, 'bc_crossover', bc_crossover_create_sql)
    print('bc_crossover created')

#bc_mutation
bc_mutation_create_sql = 'create table bc_mutation ('+ feature_create_sql +');'
try:
    createTable(mydb, 'bc_mutation', bc_mutation_create_sql)
    print('bc_mutation created')
except:
    mycursor.execute('drop table bc_mutation')
    createTable(mydb, 'bc_mutation', bc_mutation_create_sql)
    print('bc_mutation created')

bc_population created
bc_selection created
bc_crossover created
bc_mutation created


<h1> Initialise Population

In [36]:
# from compiler.ast import flatten
all_population = [list(combinations(features, i)) for i in range(1, len(features)+1)]
temp = list()
for lst in all_population:
    temp.extend(lst)
all_population = temp

all_population[:10]

MemoryError: 

In [12]:
len(all_population)

255

In [13]:
init_pop = list()
zeros = [0 for i in range(len(features))]

for pop in all_population:
    for attr in pop:
        zeros[features.index(attr)] = 1
    init_pop.append(zeros)
    zeros = [0 for i in range(len(features))]
init_pop[:10]

[[1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 0]]

In [14]:
columns = str(features).replace('[', '').replace(']', '').replace("'", '')
columns

'Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age'

In [15]:
len(init_pop)

255

<h2>Insert Population Table

In [16]:
#insert init_pop to db
insert_pop_sql = 'insert into population ('+columns+') values (%s, %s, %s, %s, %s, %s, %s, %s)'
mycursor.executemany(insert_pop_sql, init_pop)
try:
    mydb.commit()
    print(mycursor.rowcount, 'was inserted.')
except:
    mydb.rollback()
    print('rollbacked')

255 was inserted.


<h1> Evaluate and Selection

In [17]:
#get top 3/4 selection population
#the criteria to pass to the next gen is 0.5 bcuz the max value is 1 after the normalization
pass_criteria = 0.5
pass_criteria

0.5

In [18]:
history_fitness_funct = dict() # dict of zeroOneList and its score that have been used fitness_func
def fitness_func(features, zeroOneList):
    """
    mean of mahattan distance of all selected features
    """
    if str(zeroOneList) in history_fitness_funct:
        return history_fitness_funct[str(zeroOneList)]

    if zeroOneList == [0 for i in range(len(feat))]:
        return 0

    history_fitness_funct[str(zeroOneList)] = np.mean([df_md.T[f] for f in convertZeroOne(features, zeroOneList)])
    return history_fitness_funct[str(zeroOneList)]


In [19]:
selection_pop = [i for i in init_pop if fitness_func(features, i) >= 0.5]
print('Strong generation(>=',str(pass_criteria)+') :', len(selection_pop))
selection_pop

Strong generation(>= 0.5) : 250


[[1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 1],
 [0, 1, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 1, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 1],
 [1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 0, 0, 1, 0, 0, 0],
 [1, 1, 0, 0, 0, 1, 0, 0],
 [1, 1, 0, 0, 0, 0, 1, 0],
 [1, 1, 0, 0, 0, 0, 0, 1],
 

<h2>Insert Selection Table

In [20]:
#insert selection_pop to db
insert_sel_sql = 'insert into selection ('+columns+') values (%s, %s, %s, %s, %s, %s, %s, %s)'
mycursor.executemany(insert_sel_sql, selection_pop)
try:
    mydb.commit()
    print(mycursor.rowcount, 'was inserted.')
except:
    mydb.rollback()
    print('rollbacked')

250 was inserted.


<h1> Crossover

In [21]:
#random pair of parent
#random where to split the range of spliting withh be in range [round(2/8*len(features)), round(6/8*len(features))]

In [22]:
def crossover(zeroOneList1, zeroOneList2, split_point):
    """
    Crossover zeroOnelist
    """
    return [
        zeroOneList1[:split_point] + zeroOneList2[split_point:],
        zeroOneList2[:split_point] + zeroOneList1[split_point:]
    ]

In [23]:
def np_crossover(np_zeroOneList1, np_zeroOneList2, split_point):
    """
    Crossover numpy array zeroOnelist
    support np array
    """
    return np.array([
        np.concatenate([np_zeroOneList1[:split_point], np_zeroOneList2[split_point:]]),
        np.concatenate([np_zeroOneList2[:split_point], np_zeroOneList1[split_point:]])
    ])

In [24]:
len(selection_pop)

250

In [25]:
split_point = random.randrange(round(0.3*len(features)), round(0.7*len(features)))
split_point

5

In [30]:
#normal list
import time
start = time.time()

crossover_pop = list()
# selection_pop = np.array(selection_pop)
# cross = list()
for pop in selection_pop:
    for p in selection_pop:
        cross = crossover(pop, p, split_point)
        crossover_pop.append(cross[0])
        crossover_pop.append(cross[1])
crossover_pop = list(set([str(i) for i in crossover_pop]))
crossover_pop = [ast.literal_eval(i) for i in crossover_pop]
# crossover_pop = [list(i) for i in np.unique(np.array(crossover_pop), axis=0)]
print(len(crossover_pop))



end = time.time()
print('used time :', end - start)
crossover_pop[:10]

256
used time : 0.2910349369049072


[[1, 1, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0, 1],
 [0, 1, 1, 1, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 0, 0, 1],
 [1, 0, 1, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 1, 1, 0],
 [1, 1, 0, 1, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 0, 1],
 [1, 0, 1, 0, 0, 0, 0, 1]]

In [25]:
#match all population for all prob
# TODO other except binary split

# #shuffle list
# random.shuffle(selection_pop)
# #will pair by even and odd number
# if len(selection_pop)%2 == 0: #len is even
#     crossover_pop = [crossover(selection_pop[i], selection_pop[i+1], split_point) for i in range(len(selection_pop)) if i%2 == 0]
#     print('even length')
# else: #len is odd
#     crossover_pop = [crossover(selection_pop[i], selection_pop[i+1], split_point) for i in range(len(selection_pop[:-1])) if i%2 == 0]
#     crossover_pop.append(crossover(selection_pop[-2], selection_pop[-1], split_point))
#     print('odd length')
# crossover_pop[:10]

even length


[[0, 1, 0, 1, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 1],
 [1, 0, 0, 1, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 0, 1, 1, 0, 1],
 [0, 1, 1, 0, 1, 1, 0, 0],
 [0, 0, 1, 1, 1, 0, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 1],
 [0, 1, 1, 0, 1, 1, 0, 1],
 [1, 1, 0, 1, 0, 1, 0, 1]]

<h2>Insert Crossover Table

In [31]:
#insert selection_pop to db
insert_cross_sql = 'insert into crossover ('+columns+') values (%s, %s, %s, %s, %s, %s, %s, %s)'
mycursor.executemany(insert_cross_sql, crossover_pop)
try:
    mydb.commit()
    print(mycursor.rowcount, 'was inserted.')
except:
    mydb.rollback()
    print('rollbacked')

256 was inserted.


<h1> Mutation

In [32]:
def mutation(zeroOneList, muta_bit_amount):
    """
    Random flip bits of zeroOneList
    """
    flip_index_bit = list()
    while len(flip_index_bit) != muta_bit_amount:
        index = random.randrange(8)
        flip_index_bit.append(index)
        flip_index_bit = list(set(flip_index_bit))
        #got random bit index

    for i in flip_index_bit:
        if zeroOneList[i] == 1: #1
            zeroOneList[i] = 0
        else: # 0
            zeroOneList[i] = 1
    return zeroOneList

In [33]:
#mutation rate = 0.3
muta_bit_amount = random.randrange(1, round(len(features)*0.3+0.5))
muta_bit_amount

1

In [34]:
mutation_pop = [mutation(i, muta_bit_amount) for i in crossover_pop]
print(len(mutation_pop))
mutation_pop[:10]

256


[[1, 1, 0, 0, 1, 0, 1, 1],
 [1, 0, 1, 1, 0, 1, 0, 1],
 [0, 1, 1, 0, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 0, 0, 0],
 [0, 0, 1, 1, 0, 1, 0, 1],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [0, 1, 1, 1, 0, 1, 1, 0],
 [1, 1, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 1]]

<h2>Insert Mutation Table

In [35]:
#insert selection_pop to db
insert_muta_sql = 'insert into mutation ('+columns+') values (%s, %s, %s, %s, %s, %s, %s, %s)'
mycursor.executemany(insert_muta_sql, mutation_pop)
try:
    mydb.commit()
    print(mycursor.rowcount, 'was inserted.')
except:
    mydb.rollback()
    print('rollbacked')

256 was inserted.


<h1>Evaluate the top Best 3/4 of Population

In [36]:
selection_crossover_mutation = selection_pop + crossover_pop + mutation_pop
selection_crossover_mutation = list(set([str(i) for i in selection_crossover_mutation]))
selection_crossover_mutation = [ast.literal_eval(i) for i in selection_crossover_mutation]
selection_crossover_mutation[:10]

[[1, 1, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0, 1],
 [0, 1, 1, 1, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 0, 0, 1],
 [1, 0, 1, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 1, 1, 0],
 [1, 1, 0, 1, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 0, 1],
 [1, 0, 1, 0, 0, 0, 0, 1]]

In [37]:
scm_dict = dict()
for i in selection_crossover_mutation:
    scm_dict[str(i)] = fitness_func(features=features, zeroOneList=i)
#top 75% of all pop
top_75_chromosome = [ast.literal_eval(i[0]) for i in sorted(scm_dict.items(), key=lambda x: x[1], reverse=True)[:round(0.75*(len(selection_crossover_mutation)))]]
del scm_dict
print(len(top_75_chromosome))
top_75_chromosome

191


[[0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 1, 0, 0],
 [0, 1, 1, 0, 0, 1, 1, 0],
 [1, 1, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 1, 1, 0],
 [0, 1, 1, 0, 0, 1, 0, 1],
 [1, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 1, 1, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 1, 0],
 [1, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 1],
 [0, 1, 0, 1, 0, 1, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 1, 1, 0],
 [0, 1, 1, 1, 0, 1, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 1, 0, 0],
 [1, 1, 0, 0, 0, 1, 0, 0],
 [1, 1, 1, 0, 0, 1, 1, 0],
 [0, 0, 1, 0, 0, 0, 1, 0],
 [0, 1, 1, 0, 1, 1, 0, 0],
 [0, 1, 1, 1, 0, 1, 0, 1],
 [1, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 1, 1, 1],
 [1, 1, 1, 0, 0, 1, 0, 1],
 [0, 1, 0, 0, 0, 1, 0, 1],
 [0, 0, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 0, 0],
 [1, 0, 1, 0, 0, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 